In [1]:
from preprocess import *
import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# Load Image
#from PIL import Image 
#cqt_img = np.array(Image.open(file+"1_1.png").convert('L'))
#plt.imshow(cqt_img, cmap='gray')
#print(cqt_img.shape)
#print(cqt_img)

In [3]:
labels = os.listdir("../training_data/training/cqt/np")
label_indices = np.arange(1, len(labels))
to_categorical(label_indices).shape

(57, 58)

In [4]:
# Load Image, and save to npy
from PIL import Image
extension = ".png"
save_path = "/mnt/hgfs/Linux/soundAnalysis/training_data/training/cqt/np/"
img_path = "/mnt/hgfs/Linux/soundAnalysis/training_data/correct/cqt/"

def save_data_npy(filename):
    cur_file = img_path + filename + extension
    cqt_img = np.array(Image.open(cur_file).convert('L')) # Image.open(cur_file).convert('RGB')
    cqt_img = np.expand_dims(cqt_img, axis=0)
    #plt.imshow(cqt_img)
    np.save(save_path + filename, cqt_img)
    #print(cqt_img.shape)

    
### one-hot encoding
# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=save_path):
    labels = os.listdir(path)
    label_indices = np.arange(1, len(labels))
    # to_categorical: one hot encoding
    return labels, label_indices, to_categorical(label_indices)

def get_train_test(split_ratio=0.6, random_state=48):
    # Get available labels
    labels, indices, _ = get_labels(save_path)

    # Getting first arrays
    X = np.load(save_path + labels[0])
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(save_path + label)
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))
    print(X.shape[0], len(y))
    assert X.shape[0] == len(y)
    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)
"""
num=1
if num==1:
    data_path = "/mnt/hgfs/Linux/soundAnalysis/training_data/correct/cqt/np/"
    img_path  = "/mnt/hgfs/Linux/soundAnalysis/training_data/correct/cqt/"
elif num==2:
    data_path = "/mnt/hgfs/Linux/soundAnalysis/training_data/incorrect/cqt/np/"
    img_path  = "/mnt/hgfs/Linux/soundAnalysis/training_data/incorrect/cqt/"
elif num==3:
    data_path = "/mnt/hgfs/Linux/soundAnalysis/training_data/test/cqt/np/"
    img_path  = "/mnt/hgfs/Linux/soundAnalysis/training_data/test/cqt/"

for sec in range(1,30): # sec
    save_data_npy(str(num)+"_"+str(sec))
"""
# loading data
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 72, 96, 1)
X_test = X_test.reshape(X_test.shape[0], 72, 96, 1)

# one-hot encoding
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train_hot.shape)
print(y_test_hot.shape)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 72 and the array at index 1 has size 96

In [ ]:
# create Sequential model
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), input_shape = (72,96,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flatten
model.add(Flatten())
# FNN layer: 128 output
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
# Add output layer
model.add(Dense(29, activation='softmax'))
# Compile: choose loss function、optimize method
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# training, 訓練過程會存在 train_history
model.fit(X_train, y_train_hot, batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))
# 顯示損失函數、訓練成果(分數)
score = model.evaluate(X_test, y_test_hot, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
score = model.evaluate(X_test, y_test_hot, verbose=1)

# 模型存檔
from keras.models import load_model
model.save('ASR.h5')  # creates a HDF5 file 'model.h5'

In [ ]:
# prediction
mfcc = wav2mfcc('./soundfile/0601/sound1_1.wav')
mfcc_reshaped = mfcc.reshape(1, 20, 11, 1)
print("labels=", get_labels())
print("predict=", np.argmax(model.predict(mfcc_reshaped)))

In [ ]:
# preprocess.py
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
DATA_PATH = "/mnt/hgfs/Linux/soundAnalysis/training_data/correct/cqt/"


In [ ]:
# Load Image
"""
from keras.preprocessing import image
test_image = image.load_img(file+"1_1.png", target_size = (72, 96))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
"""